In [1]:
#Instalar paquete SPI - SPEI solo instalar una vez
# install.packages("SPEI")
# install.packages("readr")

In [2]:
#Cargar librería SPEI y SPEI
library(SPEI)
library(readr)


Loading required package: lmomco

Loading required package: parallel

Loading required package: ggplot2

# Package SPEI (1.7) loaded [try SPEINews()].



In [3]:
setwd("../datos/por_variables")
getwd()

[1] "/home/Carlos/Dropbox/documentosA/TFM_UNIR_2022/datos/por_variables"

## Funciones para calcular SPI  y SPEI en todas las escalas

In [4]:
spi_escalas <- function(df,escala){
  df1 = data.frame()
  df1 = cbind(df['fecha'])
  
  spi_es<-spi(df[2:length(df)], scale = escala , distribution = 'Gamma')
  
  vals=spi_es$fitted
  print (vals)
  df1$spi= vals
  return(df1)
}



# Calcular el SPEI por dos metodos 1:hargreaves, 2:thornthwaite
# con: - dfstac: datos de las estaciones
#      - escala: escala en meses
#      - dfprec: datos de precipitaciones de todas las estaciones 
#      - dftm: dependiendo del <metodo>, datos de tmax (si metodo:1) y tmed(si metodo:2)
#      - dftmin: datos de tmin de todas las estaciones
#      - metodo: metodo usado para calcular PET

spei_calc <- function(dfstac, escala, dfprec, dftm, dftmin = data.frame(), metodo = 2){
    
    dfspei <- data.frame(cbind(dfprec['fecha']))
    columnas = colnames(dfprec[,2:ncol(dfprec)])

    for (est in 1:30){
        # Asignar los datos de la estacion "est" a una Serie de Tiempo
        tspp <- ts(dfprec[est+1], end=c(2020,12), frequency=12)
        tstm = ts(dftm[est+1], end=c(2020,12), frequency=12)
        
        if (metodo == 1){
            
            tsmn = ts(dftmin[est+1], end=c(2020,12), frequency=12)
            
            # Calcular PET, por el metodo 1:hargreaves
            tspet <- hargreaves(Tmin = tsmn ,Tmax = tstm, lat = dfstac[est,3])
        
        } else {
            
            # Calcular PET, por el metodo 2:thornthwaite
            tspet <- thornthwaite(Tave = tstm, lat = dfstac[est,3])
        }
        

        # Calcular el balance hídrico: pp - PET
        tsbh = tspp - tspet

        # Calcular el SPEI para una escala "escala"
        tspei = spei(tsbh, scale = escala , distribution = 'log-Logistic')
        
        # Generar el nombre de la columna
        columna <- paste('spei', substr(columnas[est], 2, 4), sep = '')
        
        # Guardar en los valores obtenidos en dfspei
        fitted_tspei <- tspei$fitted
        dfspei[columna] <- fitted_tspei[,1]
        
    }
    
    return (dfspei)
}

### Cargar Dataframe de precipitación

In [5]:

#cargar Dataframe de precipitación de todas las estaciones 
df<-read.csv("chirps_mensual_pp_bc.csv", header = TRUE, sep = ",", dec = ".") 
res=spi_escalas(df,12)
write.csv(res, file = "../spi_spei/indices_spi12.csv", row.names = FALSE) # guarda un archivo csv
res

               X100         X101         X102         X104         X105
Jan  1           NA           NA           NA           NA           NA
Feb  1           NA           NA           NA           NA           NA
Mar  1           NA           NA           NA           NA           NA
Apr  1           NA           NA           NA           NA           NA
May  1           NA           NA           NA           NA           NA
Jun  1           NA           NA           NA           NA           NA
Jul  1           NA           NA           NA           NA           NA
Aug  1           NA           NA           NA           NA           NA
Sep  1           NA           NA           NA           NA           NA
Oct  1           NA           NA           NA           NA           NA
Nov  1           NA           NA           NA           NA           NA
Dec  1  1.122920665  1.003473549  1.001897184  0.744543465  0.460405693
Jan  2  1.331620090  1.012403397  1.012682455  0.820930959  0.45

fecha,spi
<chr>,"<mts[,30]>"
1981-01-01,"NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA"
1981-02-01,"NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA"
1981-03-01,"NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA"
1981-04-01,"NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA"
1981-05-01,"NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA"
1981-06-01,"NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA"
1981-07-01,"NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA"
1981-08-01,"NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA"
1981-09-01,"NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA"


## SPEI

In [6]:
precip<-read.csv("chirps_mensual_pp_bc.csv", header = TRUE, sep = ",", dec = ".") 

t_max<-read.csv("nasa_mensual_tmax.csv", header = TRUE, sep = ",", dec = ".")
t_med<-read.csv("nasa_mensual_tmed.csv", header = TRUE, sep = ",", dec = ".")
t_min<-read.csv("nasa_mensual_tmin.csv", header = TRUE, sep = ",", dec = ".") 

estacion<-read.csv("../formato5_chirps/stations.txt", header = FALSE, sep = ",", dec = ".") 



In [7]:
# datospei = spei_calc(estacion, 3, precip, t_max, t_min, 1)
# write.csv(datospei, file = "../spi_spei/indices_spei3.csv", row.names = FALSE) # guarda un archivo csv
# datospei

In [8]:
datospei2 = spei_calc(estacion, 12, precip, t_med)
write.csv(datospei2, file = "../spi_spei/indices_spei12.csv", row.names = FALSE)
datospei2

fecha,spei100,spei101,spei102,spei104,spei105,spei107,spei108,spei109,spei110,⋯,spei124,spei125,spei126,spei200,spei201,spei202,spei203,spei204,spei205,spei206
<chr>,<ts>,<ts>,<ts>,<ts>,<ts>,<ts>,<ts>,<ts>,<ts>,⋯,<ts>,<ts>,<ts>,<ts>,<ts>,<ts>,<ts>,<ts>,<ts>,<ts>
1981-01-01,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1981-02-01,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1981-03-01,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1981-04-01,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1981-05-01,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1981-06-01,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1981-07-01,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1981-08-01,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
1981-09-01,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
